In [ ]:
from multiprocessing import Pool
import itertools
import pandas as pd
import numpy as np


#scoring function
def calculate_element_score3(match_award, mismatch_penalty, gap_penalty, seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
    
    max_score = 0        # initial maximum score in DP table
    # Calculate DP table

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)
    
    return final_score

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty
    

#set other parameters
match_award      = 2
mismatch_penalty = -1
gap_penalty      = -1 # both for opening and extanding




def f(s):
    
    scorelist = []
    
    with open('temp.txt', 'r') as file2:
        ele = file2.readline()
        
        
    score = calculate_element_score3(match_award, mismatch_penalty, gap_penalty, ele, s)

    return score




if __name__ == '__main__':
    
    
    #load data

    df = pd.read_csv('final file hopefully no errors.csv')

    dfp = df.loc[df['Gen'] == 'ssRNA(+)']
    dfn = df.loc[df['Gen'] == 'ssRNA(-)']

    dfp = dfp.reset_index(drop = True)
    dfn = dfn.reset_index(drop = True)
    
    dfp = dfp[:15]
    dfn = dfn[:15]
    
    
    #make windows

    winlist6 = []

    for i in range(6,7):

        for i in (itertools.product('ATGC', repeat=i)):
            winlist6.append(''.join(i))
    
    
    winlist = winlist6[:20]
    
    
    #two dataframes 
    df = pd.concat([dfp,dfn])
    resultdf = pd.DataFrame(index = list(range(len(df))), columns = winlist)

      
    

    
    for element in winlist:
        pool = Pool(processes=2)
        
        with open('temp.txt', 'w') as file:
            file.write(element)
            
        d = (df['Seq'].to_list())
        
        result = pool.map(f, d)
        
        
        resultdf[element] = result
        
        
    resultdf.to_csv('win6 test.csv', index = False)



In [11]:
from multiprocessing import Pool
import itertools
import pandas as pd
import numpy as np

#scoring function
def calculate_element_score3(match_award, mismatch_penalty, gap_penalty, seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
    
    max_score = 0        # initial maximum score in DP table
    # Calculate DP table

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)
    
    return final_score

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty
    

#set other parameters
match_award      = 2
mismatch_penalty = -1
gap_penalty      = -1 # both for opening and extanding




#load data

df = pd.read_csv('final file hopefully no errors.csv')

dfp = df.loc[df['Gen'] == 'ssRNA(+)']
dfn = df.loc[df['Gen'] == 'ssRNA(-)']

dfp = dfp.reset_index(drop = True)
dfn = dfn.reset_index(drop = True)

dfp = dfp[:15]
dfn = dfn[:15]

df = pd.concat([dfp,dfn]).reset_index(drop = True)


#make windows

winlist6 = []

for i in range(6,7):

    for i in (itertools.product('ATGC', repeat=i)):
        winlist6.append(''.join(i))

window_sizes = range(6,7)

#set other parameters
match_award      = 2
mismatch_penalty = -1
gap_penalty      = -1 # both for opening and extanding

#complete workflow

for win in window_sizes:
    winlist = [w for w in winlist6 if len(w) == win]
    winlist = winlist[:20]
    
    resultdf = pd.DataFrame(index = list(range(len(df))), columns = winlist)            
    print('Done making %d windows' % len(winlist)) 
    
    c = 1
    
    for element in winlist:
        for i, r in df.iterrows():
            score = calculate_element_score3(match_award, mismatch_penalty, gap_penalty, element, r['Seq'])
            
            resultdf.loc[i, element] = score
            
        print('%d iterations done in winlist. %d remaining' % (c, (len(winlist) - c) ))
        
        c+=1

    resultdf.to_csv('win6 test2.csv' , index = False)

Done making 20 windows
1 iterations done in winlist. 19 remaining
2 iterations done in winlist. 18 remaining
3 iterations done in winlist. 17 remaining
4 iterations done in winlist. 16 remaining
5 iterations done in winlist. 15 remaining
6 iterations done in winlist. 14 remaining
7 iterations done in winlist. 13 remaining
8 iterations done in winlist. 12 remaining
9 iterations done in winlist. 11 remaining
10 iterations done in winlist. 10 remaining
11 iterations done in winlist. 9 remaining
12 iterations done in winlist. 8 remaining
13 iterations done in winlist. 7 remaining
14 iterations done in winlist. 6 remaining
15 iterations done in winlist. 5 remaining
16 iterations done in winlist. 4 remaining
17 iterations done in winlist. 3 remaining
18 iterations done in winlist. 2 remaining
19 iterations done in winlist. 1 remaining
20 iterations done in winlist. 0 remaining
